In [1]:
%run ../../modulos.ipynb

# Conexión a mongo

In [12]:
client=pymongo.MongoClient('mongodb://localhost:27017/')
db_Lichess=client['Lichess'] # database
usuarios_col=db_Lichess['usuarios'] # colección

# Obtener datos usuarios



In [2]:
def request_github(url,personal_access=personal_access_token):
    headers = {"Authorization": f"Bearer {personal_access}"}

    res=requests.get(url,headers=headers)
    return res.json()

In [3]:
def obtener_data_lichess(url):    
    headers = {"Authorization": f"Bearer {personal_access_token}"}    
    response = requests.get(url, headers=headers)    
    return response

def obtener_equipos_populares(url):    
    data = obtener_data_lichess(url)
    return data

In [4]:
grupos_unam_url = 'https://lichess.org/api/team/search?text=UNAM'
data = obtener_equipos_populares(grupos_unam_url).json()

In [6]:
club_prometeo = data['currentPageResults'][4]
id_club_prome = club_prometeo['id']

## Obtención de usuarios

In [7]:
data = requests.get('https://lichess.org/api/team/club-prometeo/users')
usuario_raw = data.text.split('\n')[:-1]
list_usuarios = list(map(json.loads, usuario_raw))

# Obtentencion de estadistica

In [20]:
def obtener_estadisticas(usuario, perf="blitz"):
    data = requests.get(f'https://lichess.org/api/user/{usuario}/perf/{perf}')
    return data.json()    

In [32]:
# Consulta para el ide del usuario

# Estadicar para partidas blitz
username = list_usuarios[0]['username']
perf = 'blitz'
estadisticas1 = obtener_estadisticas(username, perf)

In [33]:
oponentes_fuertes = estadisticas1['stat']['bestWins']['results'][:3]
ids_oponentes = [oponente['opId']['id'] for oponente in oponentes_fuertes]
ids_oponentes
obtener_estadisticas(ids_oponentes[2])

{'user': {'name': 'vladgert'},
 'perf': {'glicko': {'rating': 1427.09,
   'deviation': 80.32,
   'provisional': False},
  'nb': 4292,
  'progress': 5},
 'rank': None,
 'percentile': 40.9,
 'stat': {'_id': 'vladgert/2',
  'userId': {'id': 'vladgert', 'name': 'vladgert', 'title': None},
  'perfType': {'key': 'blitz', 'name': 'Blitz'},
  'highest': {'int': 1455,
   'at': '2021-11-09T18:44:16.284Z',
   'gameId': 'aNBQGvw2'},
  'lowest': {'int': 993,
   'at': '2019-11-14T21:06:43.174Z',
   'gameId': 'h8XAYi6A'},
  'bestWins': {'results': [{'opInt': 1546,
     'opId': {'id': 'achitescu', 'name': 'achitescu', 'title': None},
     'at': '2021-10-25T18:12:00.945Z',
     'gameId': '2MwbNJNM'},
    {'opInt': 1525,
     'opId': {'id': 'zen_knight', 'name': 'Zen_Knight', 'title': None},
     'at': '2021-05-20T13:49:58.008Z',
     'gameId': 'kd8bTsA9'},
    {'opInt': 1522,
     'opId': {'id': 'nassa2012', 'name': 'NaSSa2012', 'title': None},
     'at': '2020-12-12T13:15:16.569Z',
     'gameId': 'ewV

In [61]:
# Limpiar registros
usuarios_col.drop()
len(list(usuarios_col.find()))

0

# Neo4j

`docker run --name "Lichess_neo" -p 7474:7474 -p 7687:7687 neo4j `

In [59]:
from neo4j import GraphDatabase

data_base_connection = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "1234"))
session = data_base_connection.session() 

In [60]:
# elimina toda la base de datos
session.run('MATCH (n) DETACH DELETE n')
q=session.run('MATCH (n) RETURN n')


In [62]:

# Guardarlo en mongo
for usuario in tqdm_notebook(list_usuarios):
  id_usuario=usuario['id']
  instruccion=f'''
MERGE (u:Usuario {{id:"{id_usuario}"}})
'''
  usuarios_col.insert_one({id_usuario:usuario})
  est_usuario=obtener_estadisticas(id_usuario)
  usuarios_col.update_one({ id_usuario : { '$exists' : 'true' } } ,{'$set': est_usuario})

  # oponentes
  oponentes_fuertes = est_usuario['stat']['bestWins']['results'][:]
  ids_oponentes = [oponente['opId']['id'] for oponente in oponentes_fuertes]


  j=0
  i=0
  while i<len(oponentes_fuertes) and j<3:
  # for id_op in ids_oponentes:
    
    id_op=ids_oponentes[i]
    
    i=i+1
    try:
      est_usuario1=obtener_estadisticas(id_op)
    except: 
      continue  
    j=j+1
    usuarios_col.insert_one({id_op:''})
    usuarios_col.update_one({ id_op : { '$exists' : 'true' } } ,{'$set': est_usuario1})
    instruccion+=f'''
    MERGE (u{i}:Usuario {{id:"{id_op}"}})
    MERGE (u)-[:gano_a]->(u{i})
  '''
    #op de oponente
    oponentes_fuertes1 = est_usuario1['stat']['bestWins']['results'][:3]
    ids_oponentes1 = [oponente['opId']['id'] for oponente in oponentes_fuertes1]
    for k,id_ops_de_ops in enumerate(ids_oponentes1):
      try:
        est_usuario2=obtener_estadisticas(id_ops_de_ops)
      except: 
        continue   
      usuarios_col.insert_one({id_ops_de_ops:''})
      usuarios_col.update_one({ id_ops_de_ops : { '$exists' : 'true' } } ,{'$set': est_usuario2})
      instruccion+=f'''
    MERGE (u{i}{k}:Usuario {{id:"{id_ops_de_ops}"}})
    MERGE (u{i})-[:gano_a]->(u{i}{k})
  '''

  session.run(instruccion)


C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/87 [00:00<?, ?it/s]

In [74]:
for usuario in tqm(list_usuarios):
  id_usuario=usuario['id']
  instruccion=f'''
MERGE (u:Usuario {{id:"{id_usuario}"}})
MERGE (e:Equipo {{nombre:"club_prometeo"}})
MERGE (u)-[:pertenece]->(e)
'''
  session.run(instruccion)

  0%|          | 0/87 [00:00<?, ?it/s]

In [75]:
instruccion='''match (u1:Usuario)
match (u2:Usuario)
match (u3:Usuario)
match (u1)-[:gano_a]->(u2)-[:gano_a]->(u3)
return u3,u1'''

a=session.run(instruccion)

In [80]:
data=usuarios_col.find_one({'rockisland':{'$exists':'true'}})

In [82]:
data['stat']

{'_id': 'rockisland/2',
 'userId': {'id': 'rockisland', 'name': 'RockIsland', 'title': None},
 'perfType': {'key': 'blitz', 'name': 'Blitz'},
 'highest': {'int': 2006,
  'at': '2021-08-19T20:32:13.411Z',
  'gameId': 'MwMlR1NK'},
 'lowest': {'int': 1574,
  'at': '2019-07-16T09:46:08.580Z',
  'gameId': 'm9HJe8dQ'},
 'bestWins': {'results': [{'opInt': 2432,
    'opId': {'id': 'toto4300', 'name': 'toto4300', 'title': None},
    'at': '2021-07-31T16:12:53.527Z',
    'gameId': 'bXQd3qnU'},
   {'opInt': 2411,
    'opId': {'id': 'will-i-am_21', 'name': 'will-i-am_21', 'title': None},
    'at': '2021-08-02T18:58:51.184Z',
    'gameId': 'H8pYKWHe'},
   {'opInt': 2253,
    'opId': {'id': 'betelgeuse888', 'name': 'Betelgeuse888', 'title': None},
    'at': '2021-08-03T15:44:18.091Z',
    'gameId': 'K72xSvoi'},
   {'opInt': 2111,
    'opId': {'id': 'alex646464', 'name': 'Alex646464', 'title': None},
    'at': '2021-07-30T09:26:48.967Z',
    'gameId': '6thuCPYx'},
   {'opInt': 2066,
    'opId': {'id'

In [78]:
a.data()

[{'u3': {'id': 'rockisland'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'munjo123'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'sargan7'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'patrickmilen'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'nadir_sofiani'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'achitescu'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'zen_knight'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'nassa2012'}, 'u1': {'id': 'alanmezal'}},
 {'u3': {'id': 'kanachess'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'mattescooper89'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'peterpanter'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'titlewinner'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'killer_programs'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'zombyman'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'kon_stan'}, 'u1': {'id': 'xngelguale'}},
 {'u3': {'id': 'schachschneider'}, 'u1': {'id': 'quarknewton'}},
 {'u3': {'id': 'chess_earth'}, 'u1': {'id': 'quarknewt

# Consultas

- 5 becarios más prometedores